In [ ]:
import numpy as np
import mne
import pickle
from mne.preprocessing import read_ica
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.svm import SVC, SVR

from tqdm import tqdm
from matplotlib import pyplot as plt

from continuous_control_bci.visualisation.erds import plot_tfr
from continuous_control_bci.util import emg_classes_to_eeg_classes, SUBJECT_IDS
from continuous_control_bci.data.load_data import load_calibration, load_driving
from continuous_control_bci.data.preprocessing import make_epochs, epochs_to_train_test
from continuous_control_bci.modelling.csp_classifier import create_csp_classifier, get_driving_epochs_for_csp
from sklearn.metrics import f1_score
from filterbank_csp import filter_bank, filter_bank_csp, predict_against_threshold, predict_against_threshold_indiv, create_csp_svm_classifier, majority_pred

# This notebook tries to find if there is a difference in the Patterns found in Calibration and Driving data

In [ ]:
subject_id = "061"
include_rest = True
if include_rest:
    target_names = ["Left", "Right", "Rest"]
else:
    target_names = ["Left", "Right"]
raw = load_calibration(subject_id)
raw.set_eeg_reference()
raw.filter(l_freq=5, h_freq=35)
eeg_channel_indices = mne.pick_types(raw.info, eeg=True, emg=False)
ica = read_ica(f'D:/RUG/Term 2/FYRP/data/ica/P{subject_id}-calibration-ica.fif')
ica.apply(raw)
epochs = make_epochs(raw, include_rest=include_rest)
X_train, _, y_train, _ = epochs_to_train_test(epochs)

In [ ]:
epochs.compute_psd(fmin=6, fmax=15, exclude='bads', picks='eeg', method='welch').plot(average=True)